In [1]:
import warnings
warnings.filterwarnings("ignore")


In [2]:
import sys
sys.path.append("..\src")
import pandas as pd
from paths import TRANSFORMED_DATA_DIR
df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,11.0,15.0,26.0,8.0,9.0,7.0,3.0,1.0,0.0,3.0,...,11.0,7.0,4.0,3.0,4.0,9.0,19.0,2022-01-29,4,17.0
1,1.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,5.0,4.0,10.0,7.0,5.0,9.0,10.0,2022-01-30,4,9.0
2,0.0,1.0,0.0,0.0,1.0,1.0,1.0,3.0,2.0,3.0,...,8.0,7.0,8.0,5.0,5.0,10.0,0.0,2022-01-31,4,3.0
3,1.0,1.0,0.0,0.0,0.0,3.0,2.0,3.0,4.0,5.0,...,3.0,16.0,7.0,1.0,0.0,1.0,3.0,2022-02-01,4,3.0
4,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,1.0,2.0,...,3.0,8.0,3.0,0.0,4.0,4.0,3.0,2022-02-02,4,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-27,199,0.0
88290,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-28,199,0.0
88291,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-29,199,0.0
88292,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-12-30,199,0.0


In [3]:
from datetime import datetime
from data_split import  train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=datetime(2022, 7, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(40086, 674)
y_train.shape=(40086,)
X_test.shape=(48208, 674)
y_test.shape=(48208,)


In [4]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from model import get_pipeline





def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on a TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),   
    }
       
    tss = TimeSeriesSplit(n_splits=4)
    scores = []
    
    # sort X_train by `pikup_hour` inplace
    # so the TimeSeriesSplit will split the data in a consistent way
    X_train.sort_values('pickup_hour', inplace=True)

    for train_index, val_index in tss.split(X_train):

        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index,:]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
            # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)

        scores.append(mae)
   
    # Return the mean score
    return np.array(scores).mean()

In [5]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=5)

[I 2024-02-16 16:59:09,564] A new study created in memory with name: no-name-e96474ab-3f2d-4ba2-aff8-4b72f1b30c0a
[I 2024-02-16 16:59:29,614] Trial 0 finished with value: 23.835415464862372 and parameters: {'num_leaves': 10, 'feature_fraction': 0.42993476862453045, 'bagging_fraction': 0.9509936966124644, 'min_child_samples': 27}. Best is trial 0 with value: 23.835415464862372.
[I 2024-02-16 17:00:58,166] Trial 1 finished with value: 23.907624459606534 and parameters: {'num_leaves': 180, 'feature_fraction': 0.9880715373274707, 'bagging_fraction': 0.506407604114506, 'min_child_samples': 71}. Best is trial 0 with value: 23.835415464862372.
[I 2024-02-16 17:02:30,654] Trial 2 finished with value: 23.926478993056293 and parameters: {'num_leaves': 232, 'feature_fraction': 0.6476295076265567, 'bagging_fraction': 0.24770328625535126, 'min_child_samples': 30}. Best is trial 0 with value: 23.835415464862372.
[I 2024-02-16 17:03:15,898] Trial 3 finished with value: 23.57676180301512 and parameter

In [8]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 154, 'feature_fraction': 0.42040324707773735, 'bagging_fraction': 0.7544936138234537, 'min_child_samples': 66}


In [9]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.42040324707773735, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42040324707773735
[LightGBM] [Warning] bagging_fraction is set=0.7544936138234537, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7544936138234537
[LightGBM] [Warning] feature_fraction is set=0.42040324707773735, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42040324707773735
[LightGBM] [Warning] bagging_fraction is set=0.7544936138234537, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7544936138234537
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.302228 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 158462
[LightGBM] [Info] Number of data points in the train set: 40086, number of used features: 675
[LightGBM] [Info] Start training from score 11.973233


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x0000026CFF376340>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.7544936138234537,
                               feature_fraction=0.42040324707773735,
                               min_child_samples=66, num_leaves=154))])

In [10]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.42040324707773735, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.42040324707773735
[LightGBM] [Warning] bagging_fraction is set=0.7544936138234537, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7544936138234537
test_mae=19.6349


In [87]:
from plot import plot_one_sample

plot_one_sample(
    example_id=54,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)

In [86]:
plot_one_sample(
    example_id=130,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)